In [1]:
import pandas as pd
import numpy as np

import torch
from torch_geometric.data import InMemoryDataset, Data

RDKit WARNING: [09:15:36] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
class PeMSD7M(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super(PeMSD7M, self).__init__(root, transform, pre_transform, pre_filter)
        
        # load processed data
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return ['V_228_modified.csv', 'W_228.csv']
    
    @property
    def processed_file_names(self):
        return ['data.pt']
    
    def process(self):
        pemsd7_v = pd.read_csv(self.raw_paths[0], header=None)
        pemsd7_w = pd.read_csv(self.raw_paths[1], header=None)
        pemsd7_v = pemsd7_v.to_numpy()
        pemsd7_w = pemsd7_w.to_numpy()
        data_list = []
        
        nodes = torch.tensor([i for i in range(228)],
                     dtype=torch.float).unsqueeze(1)
        
        edge_index = torch.tensor([[j, i] for j in range(228) for i in range(228) if i!=j],
                   dtype=torch.long)
        
        edge_weight = []
        for k in edge_index:
            i, j = k
            edge_weight = np.append(edge_weight, pemsd7_w[i, j])
        edge_weight = torch.tensor(edge_weight, dtype=torch.float).unsqueeze(1)
        print('In process')
        length = pemsd7_v.shape[0]
        for i in range(length):
            node_feature = torch.tensor(pemsd7_v[i], dtype=torch.float).unsqueeze(1)
            data = Data(x=node_feature,
                        edge_index = edge_index.T,
                        edge_weight = edge_weight)
            data_list.append(data)
        print('In process 2')
        print(len(data_list))
        self.data, self.slices = self.collate(data_list[0:2880])
        print('In process 3')
        torch.save((self.data, self.slices), self.processed_paths[0])
        print('In process 4')

In [3]:
traffic = PeMSD7M(root='./data/PeMS-M/')

In [4]:
traffic

PeMSD7M(2880)

In [5]:
# import networkx as nx
# from torch_geometric import utils

# G = utils.to_networkx(traffic[0])
# nx.draw(G)

In [30]:
from data_utils import describe_data, load_data
from layer import TemporalConvGLU, SpatialConv

# Loading data

In [7]:
describe_data(traffic)
train, val, test = load_data(traffic)
print('Data Loading Complete!')
print(len(train), len(val), len(test))


Dataset: PeMSD7M(2880):
Number of graphs: 2880
Number of features: 1

Data(edge_index=[2, 51756], edge_weight=[51756, 1], x=[228, 1])
Number of nodes: 228
Number of edges: 51756
Average node degree: 227.00
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True
28
187 24 30
Data Loading Complete!
187 24 30


In [8]:
print(train)

In [9]:
for batch in train:
    for i, j in enumerate(batch):
        print(i, j[0], j[1][1].shape)
        print()
    break

0 batch torch.Size([])

1 edge_index torch.Size([621072])

2 edge_weight torch.Size([1])

3 x torch.Size([1])



In [10]:
for batch in train:
    print(batch.x.shape)
    break

torch.Size([2736, 1])


In [11]:
from torch_geometric.nn import GCNConv

model = GCNConv(in_channels=1, out_channels=1)

In [49]:
print(traffic[63].x.shape, traffic[63].edge_index.shape)
# out = model(x=traffic[63].x, edge_index=traffic[63].edge_index)
print(batch.x.shape, batch.edge_index.shape)
out = model(x=batch.x, edge_index=batch.edge_index)
print(out.shape)

torch.Size([228, 1]) torch.Size([2, 51756])
torch.Size([2736, 1]) torch.Size([2, 621072])
torch.Size([2736, 1])


In [13]:
out

tensor([[-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],
        [-35.6974],


In [14]:
batch.x[:228].shape

torch.Size([228, 1])

In [25]:
import torch.nn as nn

In [36]:
def TemporalConv1(x, K_t=3, c_in=1, c_out=2):
    '''
    location = layers.py
    x : input list of 'Data' objects.
    K_t: temporal kernel size = 3
    c_in: input channels
    c_out: output channels
    '''
    # num_nodes = x[0].num_nodes
    # node_feature_list = []

    # for graph in range(len(x)):
    #     node_feature_list.append(x[graph].node_feature)

    # node_feature_tensor = torch.stack(node_feature_list)
    # node_feature_tensor = node_feature_tensor.permute(1, 2, 0)
    # node_feature_tensor is now of shape [228, 1, 12]; ready for Conv1d
    print(x.shape)
    x = x.view(228, 1, 12)
    print(x.shape)
    conv1d = nn.Conv1d(c_in, c_out, K_t)
    # expected (1, 1, 3)
    return conv1d(x)

In [39]:
out = TemporalConv(x=batch.x,K_t=3, c_in=1, c_out=1)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [1, 1, 3], but got 2-dimensional input of size [2736, 1] instead

In [28]:
out

tensor([[[-32.3723, -31.6099, -32.0310,  ..., -31.5583, -21.3986, -32.1188]],

        [[-33.8266, -31.2941, -33.7482,  ..., -32.2624, -33.8605, -33.9300]],

        [[-31.9613, -33.6791, -31.9438,  ..., -32.1032, -34.1020, -33.2194]],

        ...,

        [[-31.0930, -34.6027, -31.2607,  ..., -32.0144, -33.8522, -31.9044]],

        [[-31.4580, -31.7770, -32.8877,  ..., -31.5841, -31.6563, -32.9259]],

        [[-32.5528, -32.4739, -34.0157,  ..., -30.7514, -30.1212, -30.4551]]],
       grad_fn=<SqueezeBackward1>)

In [42]:
from layers import TemporalConvGLU

In [44]:
temp1 = TemporalConvGLU(x=batch.x,K_t=3, c_in=1, c_out=1)
# spat1 = SpatialConv(K_s=1, c_in=1, c_out=1)
# temp2 = TemporalConvGLU(K_t=3, c_in=1, c_out=1)

torch.Size([2736, 1])
torch.Size([228, 1, 12])


AttributeError: module 'torch.functional' has no attribute 'glu'